# Intraclass correlation statistics to test item performance models


## Two ways of correlation estimates:

- Analysis of Variance (ANOVA)
- Permutation Resambling - based on Monte Carlo Method

## ANOVA vs Permutation Resambling (advantages and disadvantages):

ANOVA:
- fast
- accurate
- provides reliable confidence limits for the ICC
- assumes the underlying ariables are approximately Gaussian
- less sensitive to missing data than Permutation Resambling

Permutation Resambling:
- distribution free
- highly flexible
- requires much more computational effort than ANOVA
- more sensitive to missing data than ANOVA

## Standard analysis of variance

Three variation sources:
- **the between rows item effect**: 

$MSi$ - mean square

$dfi = m - 1$ - degrees of freedom

$E(MSi) = n\sigma_{\beta}^{2} + \sigma_{\epsilon}$ - expected value

- **the between-column participant effect**:

$MSp$ - mean square

$dfp = n - 1$ - degrees of freedom

$E(MSe) = m\sigma_{\alpha}^{2} + \sigma_{\epsilon}$ - expected value

- **the residual error effect**:

$MSe$ - mean square

$dfp = (n - 1)(m - 1) = N - 1 - dfi - dfp$ - degrees of freedom

where:

N - total number of available measures in the matrix

$E(MSe) = \sigma_{\epsilon}$ - expected value

**Estimate of the q-ratio**:
$$ \hat{q}  = \frac{MSi - MSe}{nMSe}$$

**Estimate of the interclass coefficient**:
$$\frac{n\hat{q}}{n\hat{q} + 1} = \frac{MSi - MSe}{MSi}$$

**The confidence interval of probability $1 - \alpha$**:
$$\Bigg[1 - \frac{F_{1 - \alpha/2}(dfi, dfe)}{F_{obs}}; 1 - \frac{1}{F_{obs} x F_{1 - \alpha/2}(dfe, dfi)}\Bigg]$$

where:

$F_{obs}= MSi/MSe$

$F_{p}(a, b)$ - quantile of probability of p of Fisher $F$ distribution with $a$ (numerator) and $b$ (donominator) degrees of freedom

### Interclass correlation coefficient interpretatin 
[Source: https://en.wikipedia.org/wiki/Intraclass_correlation]:

Guidelines for interpretation proposed by Cicchetti (1994):
- less than 0.40—poor
- between 0.40 and 0.59—fair
- between 0.60 and 0.74—good
- between 0.75 and 1.00—excellent

Guidelines for interpretation proposed by Koo and Li (2016):
- below 0.50: poor
- between 0.50 and 0.75: moderate
- between 0.75 and 0.90: good
- above 0.90: excellent

### Loading data

In [1]:
# import all requaired packages

import pandas as pd
import numpy as np
import scipy.stats as stats # for calculate interval for ICC

In [2]:
#loading all datasets

# Respond Time 1 database
RT1 = pd.read_csv('/Users/klaudiaszczygiel/data_science/Applied_statistics/ERPbox/RTdataB1.csv', header=None)
RT_1 = RT1.copy() # for easier computation later

# Respond Time 2 database
RT2 = pd.read_csv('/Users/klaudiaszczygiel/data_science/Applied_statistics/ERPbox/RTdataB2.csv', header=None)
RT_2 = RT2.copy() 

# Respond Time 3 database
RT3 = pd.read_csv('/Users/klaudiaszczygiel/data_science/Applied_statistics/ERPbox/RTdataB3.csv', header=None)
RT_3 = RT3.copy() 

# Respond Time 4 database
RT4 = pd.read_csv('/Users/klaudiaszczygiel/data_science/Applied_statistics/ERPbox/RTdataB4.csv', header=None)
RT_4 = RT4.copy() 

# naming2013 database
df2013 = pd.read_csv('/Users/klaudiaszczygiel/data_science/Applied_statistics/ERPbox/naming2013.csv', header=None)
df_2013 = df2013.copy() # for easier computation later

## 1. Respond Time 1 database (RT1)

- detailed case, described step by step compution 

In [3]:
# quick overview

RT1.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,516.0,430.0,453,449,431,448,434.0,411.0,473.0,527,...,NaN,519.0,597,NaN,541,420.0,479.0,523.0,474.0,456.0
1,278.0,NaN,354,467,607,428,493.0,NaN,509.0,405,...,530.0,411.0,346,440.0,473,NaN,415.0,480.0,411.0,432.0
2,419.0,379.0,421,412,440,344,397.0,435.0,429.0,395,...,483.0,392.0,415,NaN,422,416.0,437.0,410.0,502.0,404.0
3,425.0,406.0,429,480,447,482,439.0,461.0,417.0,375,...,600.0,597.0,468,585.0,621,379.0,449.0,661.0,548.0,586.0
4,460.0,373.0,511,555,517,458,470.0,479.0,583.0,452,...,622.0,457.0,468,553.0,516,495.0,572.0,547.0,648.0,490.0


### List of lists from data

At the beginning to improve and simplyfy the calculations we convert dataframe into matrix (direct matrices in python are not avaible but we can skip this problem by building list of lists from our datasets which works similar to the matrices). Next we transpose our matrices because we want to have `m` rows (nb of items) and `n` columns (nb of participants).

**Desired input**:

`m` : nb of items in the items populations

`n` : nb of participants in the participants population

`m x n` : matrix of behavioural measures

In [4]:
RT1 = np.array(RT1.values.tolist()).transpose()

In [5]:
# quick overview - seems to look properly, now row are responsible for the items and the columns for participants
RT1

array([[516., 278., 419., ...,  nan, 275., 541.],
       [430.,  nan, 379., ..., 276., 415., 605.],
       [453., 354., 421., ..., 287., 564., 624.],
       ...,
       [523., 480., 410., ..., 469., 582., 479.],
       [474., 411., 502., ..., 459., 350., 552.],
       [456., 432., 404., ..., 482., 518., 502.]])

### Single functions required for computing ICC

- to better understanding how to caompute components for ANOVA
- functions writes for RT1 databeses

In [ ]:
m = 200
n = 48

In [ ]:
# t_j: sum the total respond time for each participants (function for double loop)
t_j = []
for i in range(m):
    for j in range(n):
        t_j.append(np.nansum(RT1.transpose()[:][j]))
#t_j[:48]

In [ ]:
# t_j: sum the total respond time for each participants (sigle function)

def participant_time_sum():
    '''Return list which contain sum of respond time for each participant (sums value for each column)'''
    t_j = []
    for j in range(n):
        t_j.append(np.nansum(RT1.transpose()[:][j]))
    return t_j

In [ ]:
# t_i: sums all of items value in rows

def item_sum():
    '''Return list which contain sum of respond time for each item (sums value for each row).'''
    t_i = []
    for i in range(m):
        t_i.append(np.nansum(RT1[i][:]))
    return t_i

In [ ]:
# mitem:  return list of means for each item, work well: nanmean - just skip NaN without removing rows

def mean_of_item():
    '''Return list of means for each item (calculate mean for each row).
       Deal also with missing values'''
    mitem = []
    for i in range(m):
        mitem.append(np.nanmean(RT1[i][:]))
    return mitem

In [ ]:
# sum of squares

def sum_of_squares():
    '''Return sum of squres for all values of 
       respond time. Deal also with missing values.'''
    sx2 = 0
    for i in range(m):
        for j in range(n):
            sx2 = sx2 + np.nansum(RT1[i][j]**2)
    return sx2

### All functions blend together

In [6]:
# Improved version

m = 200 # nb of words
n = 48 # nb of participants

sx2 = 0 # sum of squares
t_i = []
t_j = []
mitem = [] # list for means fro each item
for i in range(m):
    mitem.append(np.nanmean(RT1[i][:]))
    t_i.append(np.nansum(RT1[i][:]))
    for j in range(n):
        if RT1[i][j] != 'nan':
            t_j.append(np.nansum(RT1.transpose()[:][j])) # work as expected
            sx2 = sx2 + np.nansum(RT1[i][j]**2) # work as expected - checked with another method of computing

n_i = list(RT_1.apply(lambda x: x.count(), axis=0)) # totall non missing values for rows
n_j = list(RT_1.apply(lambda x: x.count(), axis=1)) # totall non missing values for columns        
N = np.sum(n_i)
t = np.sum(t_i)
ss = sx2 - t**2/N 
ssi = np.sum([x**2/y for x, y in zip(map(float, t_i), map(float, n_i))]) - t**2/N # each of elements from t_i raised to power 2 and divided by 48 and sums
ssj = np.sum([x**2/y for x, y in zip(map(float, t_j[:n]), map(float, n_j))]) - t**2/N # like higher
ssij = ss - ssi - ssj
dfi = m-1
dfj = n-1
dfij = N-1-dfi-dfj
msi = ssi/dfi
vij = ssij/dfij
vi = max(0, (msi - vij)/n)
qAV = vi/vij
icc = vi/(vi+vij/n)
F_obs = msi/vij

In [7]:
print("N:",N) # checked - ok
print("t:",t) # checked - ok
print("ss:",ss) # compute properly, checked with another method
print("ssi:",ssi) # compute properly, checked with another method
print("ssj:",ssj) # compute properly, checked with another method
print("ssij:",ssij) # compute properly, checked with another method
print("dfi:",dfi) # checked - ok
print("dfj:",dfj) # checked - ok
print("dfij:",dfij) # checked - ok
print("msi:",msi) # compute properly, checked with another method
print("vij:",vij) #check
print("vi:",vi) #chceck
print("qAV:",qAV) 
print("icc:",icc) # seems to work properly
print("F_obs:",F_obs)

N: 9166
t: 4550829.0
ss: 75469714.61629915
ssi: 5280052.288134098
ssj: 28167883.783376217
ssij: 42021778.54478884
dfi: 199
dfj: 47
dfij: 8919
msi: 26532.926071025617
vij: 4711.489914204377
vi: 454.6132532671092
qAV: 0.09649033777967433
icc: 0.82242855908194
F_obs: 5.631536213424368


### Compute the ICC confidence intervals 

In [8]:
pconf = [0.95, 0.99, 0.999]

Q1f = []
Q2f = []
for i in range(len(pconf)):
    Q1 = stats.f.ppf(1 - (1 - pconf[i])/2, dfi, dfij)
    Q2  = stats.f.ppf(1 - (1 - pconf[i])/2, dfij, dfi)
    Q1f.append(Q1)
    Q2f.append(Q2)
    print(f'Quantiles for alpha={pconf[i]}: {Q1},{Q2}')

Quantiles for alpha=0.95: 1.208720821014422,1.2321271218589724
Quantiles for alpha=0.99: 1.2811384278966609,1.3180068682994863
Quantiles for alpha=0.999: 1.3687652129202201,1.427749119290794


In [9]:
def conf():
    ''''Return confidence intervals for Fisher F-distribution'''
    conf = np.zeros((3,3))
    for i in range(len(pconf)):
        conf[i][0] = pconf[i]
        conf[i][1] = 1 - (Q1f[i]/F_obs)
        conf[i][2] = 1 - 1/(Q2f[i] * F_obs)
    return conf

In [10]:
conf()

array([[0.95      , 0.7853657 , 0.85588221],
       [0.99      , 0.7725064 , 0.86527275],
       [0.999     , 0.75694639, 0.8756284 ]])

## 2. General case for RT1, RT2, RT3, RT4, naming2013.

In [21]:
def ICC(m,n,data,data_1):
    '''Function which return ICC values with confidence interval for sigle database.'''
    sx2 = 0 # sum of squares
    t_i = []
    t_j = []
    mitem = [] # list for means fro each item
    for i in range(m):
        mitem.append(np.nanmean(data[i][:]))
        t_i.append(np.nansum(data[i][:]))
        for j in range(n):
            if data[i][j] != 'nan':
                t_j.append(np.nansum(data.transpose()[:][j])) # work as expected
                sx2 = sx2 + np.nansum(data[i][j]**2) # work as expected - checked with another method of computing

    n_i = list(data_1.apply(lambda x: x.count(), axis=0)) # totall non missing values for rows
    n_j = list(data_1.apply(lambda x: x.count(), axis=1)) # totall non missing values for columns        
    N = np.sum(n_i)
    t = np.sum(t_i)
    ss = sx2 - t**2/N 
    ssi = np.sum([x**2/y for x, y in zip(map(float, t_i), map(float, n_i))]) - t**2/N # each of elements from t_i raised to power 2 and divided by 48 and sums
    ssj = np.sum([x**2/y for x, y in zip(map(float, t_j[:n]), map(float, n_j))]) - t**2/N # like higher
    ssij = ss - ssi - ssj
    dfi = m-1
    dfj = n-1
    dfij = N-1-dfi-dfj
    msi = ssi/dfi
    vij = ssij/dfij
    vi = max(0, (msi - vij)/n)
    qAV = vi/vij
    icc = vi/(vi+vij/n)
    F_obs = msi/vij
    
    def conf():
        ''''Return confidence intervals for Fisher F-distribution'''
        pconf = [0.95, 0.99, 0.999]
        Q1f = []
        Q2f = []
        for i in range(len(pconf)):
            Q1 = stats.f.ppf(1 - (1 - pconf[i])/2, dfi, dfij)
            Q2  = stats.f.ppf(1 - (1 - pconf[i])/2, dfij, dfi)
            Q1f.append(Q1)
            Q2f.append(Q2)
    
        conf = np.zeros((3,3))
        for i in range(len(pconf)):
            conf[i][0] = pconf[i]
            conf[i][1] = 1 - (Q1f[i]/F_obs)
            conf[i][2] = 1 - 1/(Q2f[i] * F_obs)
        return conf
    
    return print(f'ICC value: {icc}'), print(f'Confidence intervals: \n{conf()}'), print('*'*40)

In [22]:
ICC(200, 48, RT1, RT_1)
ICC(200, 48, RT2, RT_2)
ICC(200, 48, RT3, RT_3)
ICC(200, 48, RT4, RT_4)
ICC(200, 100, df2013, df_2013)

ICC value: 0.82242855908194
Confidence intervals: 
[[0.95       0.7853657  0.85588221]
 [0.99       0.7725064  0.86527275]
 [0.999      0.75694639 0.8756284 ]]
****************************************
ICC value: 0.7536994056041846
Confidence intervals: 
[[0.95       0.70228492 0.80010484]
 [0.99       0.68444578 0.81313075]
 [0.999      0.66285973 0.82749523]]
****************************************
ICC value: 0.784645450992303
Confidence intervals: 
[[0.95       0.73968578 0.82522313]
 [0.99       0.72408594 0.83661301]
 [0.999      0.70520925 0.84917322]]
****************************************
ICC value: 0.750283187069434
Confidence intervals: 
[[0.95       0.69815607 0.797332  ]
 [0.99       0.6800697  0.81053852]
 [0.999      0.65818452 0.82510219]]
****************************************
ICC value: 0.8971072920273282
Confidence intervals: 
[[0.95       0.87577881 0.91641084]
 [0.99       0.86838793 0.9218349 ]
 [0.999      0.85945248 0.92782002]]
******************************

(None, None, None)